<a href="https://colab.research.google.com/github/duper203/upstage_cookbook/blob/main/financial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import userdata
upstage_api_key = userdata.get('upstage_api_key')

In [2]:
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 11.3 MB/s eta 0

In [3]:
!pip install langchain_upstage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.7 MB/s eta 0:00:00


In [4]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.3 MB/s eta 0:00:00


# INTRO
### Financial Analysis over 10-K documents
A key part of a financial analyst's job is to extract information and synthesize insight from long financial documents. A great example is the 10-K form - an annual report required by the U.S. Securities and Exchange Commission (SEC), that gives a comprehensive summary of a company's financial performance. These documents typically run hundred of pages in length, and contain domain-specific terminology that makes it challenging for a layperson to digest quickly.

We showcase how 'Upstage' can support a financial analyst in quickly extracting information and synthesize insights across multiple documents with very little coding.

## 1. Extract data from Document & Split


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain_upstage import UpstageLayoutAnalysisLoader
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings

In [6]:
# check if text is in the vector store
def is_in_vectorstore(vectorstore, text):
    search_results = vectorstore.get(ids=[text])
    if search_results and search_results["ids"]:
        return True
    else:
        return False


In [8]:
file_path = "c3_k-10.pdf"
# For image files, set use_ocr to True to perform OCR inference on the document before layout detection.
loader = UpstageLayoutAnalysisLoader(file_path, split="page", api_key=upstage_api_key, use_ocr=True)

# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = loader.load()  # or loader.lazy_load()

In [9]:
print(docs[0].page_content[:1000])


<p id='1' data-category='paragraph' style='font-size:22px'>UNITED STATES<br>SECURITIES AND EXCHANGE COMMISSION<br>Washington, D.C. 20549<br>FORM 10-K</p> <br><p id='2' data-category='paragraph' style='font-size:16px'>(Mark One)</p> <br><p id='3' data-category='paragraph' style='font-size:14px'>ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934<br>For the fiscal year ended April 30, 2022</p> <p id='4' data-category='paragraph' style='font-size:14px'>OR</p> <p id='5' data-category='paragraph' style='font-size:14px'>TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934</p> <br><p id='6' data-category='paragraph' style='font-size:14px'>For the transition period from to<br>Commission File Number: 001-39744</p> <p id='7' data-category='paragraph' style='font-size:20px'>C3.ai, Inc.<br>(Exact name of registrant as specified in its charter)</p> <p id='8' data-category='paragraph' style='font-size:14px'>Delaware<br>(State or ot

In [10]:
# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))

Splits: 985


## 2. Store in a vectordb

In [11]:
vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=UpstageEmbeddings(model="solar-embedding-1-large", upstage_api_key=upstage_api_key),
)
retriever = vectorstore.as_retriever()


unique_splits = [
    split for split in splits if not is_in_vectorstore(vectorstore, split.page_content)
]
print(len(unique_splits))

985


In [18]:
import hashlib

def generate_unique_id(content, index):
    return hashlib.md5(f"{index}-{content}".encode()).hexdigest()
if len(unique_splits) > 0:
    vectorstore = Chroma.from_documents(
        ids=[generate_unique_id(split.page_content, i) for i, split in enumerate(unique_splits)],
        persist_directory="./chroma_db",
        documents=unique_splits,
        embedding=UpstageEmbeddings(model="solar-embedding-1-large", upstage_api_key=upstage_api_key),

)

## 3. Retrive relevant docs
### E.g. About the company's Financial risks

In [13]:
search_result = retriever.invoke("Tell me about finanical risks")

doc_base = []
print(search.page_content for search in search_result)
print(search_result[0].page_content[:100])
print(search_result[0])



<generator object <genexpr> at 0x7c7c6a91fa00>
<p id='83' data-category='paragraph' style='font-size:16px'>ITEM 7A. QUANTITATIVE AND QUALITATIVE DI
page_content='<p id='83' data-category='paragraph' style='font-size:16px'>ITEM 7A. QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK</p> <p id='84' data-category='paragraph' style='font-size:14px'>We are exposed to market risks in the ordinary course of our business. Market risk represents the risk of loss that may impact our financial position due to<br>adverse changes in financial market prices and rates. Our market risk exposure is primarily the result of fluctuations in interest rates and foreign currency<br>exchange rates. We do not hold or issue financial instruments for trading purposes.</p> <h1 id='85' style='font-size:16px'>Interest Rate Risk</h1>' metadata={'page': 87}


In [14]:
from bs4 import BeautifulSoup

for search in search_result:
    print(search.page_content)
    soup = BeautifulSoup(search.page_content, 'html.parser')
    text = soup.get_text(separator="\n")
    doc_base.append(text)

<p id='83' data-category='paragraph' style='font-size:16px'>ITEM 7A. QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK</p> <p id='84' data-category='paragraph' style='font-size:14px'>We are exposed to market risks in the ordinary course of our business. Market risk represents the risk of loss that may impact our financial position due to<br>adverse changes in financial market prices and rates. Our market risk exposure is primarily the result of fluctuations in interest rates and foreign currency<br>exchange rates. We do not hold or issue financial instruments for trading purposes.</p> <h1 id='85' style='font-size:16px'>Interest Rate Risk</h1>
<p id='59' data-category='paragraph' style='font-size:16px'>SELECTED RISKS AFFECTING OUR BUSINESS</p> <p id='60' data-category='paragraph' style='font-size:16px'>Investing in our Class A common stock involves numerous risks, including the risks described under "Risk Factors" in Part 1, Item 1A of this Annual<br>Report on Form 10-K. Below 

In [20]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

llm = ChatUpstage(api_key=upstage_api_key)

prompt_template = PromptTemplate.from_template(
    """
    Generate mainly two things from the following context.
    1.Summary of Risks: Generate a summary report that includes an overview of identified risks, categorized by type, with a severity ranking.
    2. Detailed Analysis: Provide a detailed breakdown of each identified risk, including the specific language from the document, context, and potential impact.
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [21]:
chain.invoke({"Context": doc_base})


"Summary of Risks:\n\n1. Market risk exposure primarily due to fluctuations in interest rates and foreign currency exchange rates.\n2. Limited operating history, making it difficult to evaluate prospects and future results of operations.\n3. Risks related to the business and industry, including potential harm to business, financial condition, operating results, and prospects.\n4. Concentration of credit risk in cash and cash equivalents held by one financial institution, potentially exceeding FDIC insurance limits.\n\nDetailed Analysis:\n\n1. Market risk exposure: The company is exposed to market risks in the ordinary course of business, including fluctuations in interest rates and foreign currency exchange rates. However, the company does not hold or issue financial instruments for trading purposes.\n2. Limited operating history: The company has a limited operating history, which makes it difficult to evaluate its prospects and future results of operations.\n3. Risks related to the bu